In [1]:
#!pip install PyPDF2

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification #BERT
import PyPDF2 #for Pdf file
import re #Regular Expressions
import numpy as np

In [3]:
pdf = open(r"D:\MAS\Semester 2\NLP\Project\Resume Analyzer\data\data\PUBLIC-RELATIONS\10070224.pdf",'rb')


In [4]:
pdfReader = PyPDF2.PdfReader(pdf)

In [5]:
pdfReader.metadata

{'/Title': '',
 '/Creator': 'wkhtmltopdf 0.12.4',
 '/Producer': 'Qt 4.8.7',
 '/CreationDate': "D:20210808155148+05'30'"}

In [6]:
x = (len(pdfReader.pages))
print("Number of pages: ",x)

Number of pages:  2


In [7]:
text = " "

In [8]:
for page_number in range(x):
    pdfReader = PyPDF2.PdfReader(pdf)
    if pdfReader.is_encrypted:
        pdfReader.decrypt('')
    page = pdfReader.pages[page_number]
    text += page.extract_text()

In [9]:
pdf.close()

In [10]:
text

' PUBLIC RELATIONS DIRECTOR\nSummary\nTalented PR professional with 11 years\' expertise developing and executing all aspects of public outreach campaigns to increase media exposure\nand support strategic marketing initiatives. Proven creative talent with demonstrated strength in media production, messaging, communications and\ndissemination techniques and methods.Â  Deep understanding of media relations; accomplished track record of securing live television and on-site\npress coverage.\nHighlights\nPublicity and communications\nVendor and client relations\nMedia relations\nIntegrated campaigns\nProject oversight\nAdvertising\nWriting and editing \nStrategic messaging\nSpecial event coordinationÂ \nEmail marketingÂ \nCrisis communication\nJournalistic interviews \nExperience\nPublic Relations Director\n \n, \n01/2016\n \nto \nCurrent\n \nCompany Name\n \nï¼\u200b \nCity\n \n, \nState\nAssistant Director of Public Relations\n \n, \n01/2010\n \nto \n01/2016\n \nCompany Name\n \nï¼\u200b 

In [11]:
text = re.sub('http\S+\s*', ' ', text)  # remove URLs
text = re.sub('RT|cc', ' ', text)  # remove RT and cc
text = re.sub('#\S+', '', text)  # remove hashtags
text = re.sub('@\S+', '  ', text)  # remove mentions
text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', text)  # remove punctuation
text = re.sub(r'[0-9]+', '', text)
text = re.sub(r'[^\x00-\x7f]',r' ', text) 
text = re.sub('\s+', ' ', text)  # remove extra whitespace
text

' PUBLIC RELATIONS DIRECTOR Summary Talented PR professional with years expertise developing and executing all aspects of public outreach campaigns to increase media exposure and support strategic marketing initiatives Proven creative talent with demonstrated strength in media production messaging communications and dissemination techniques and methods Deep understanding of media relations a omplished track record of securing live television and on site press coverage Highlights Publicity and communications Vendor and client relations Media relations Integrated campaigns Project oversight Advertising Writing and editing Strategic messaging Special event coordination Email marketing Crisis communication Journalistic interviews Experience Public Relations Director to Current Company Name City State Assistant Director of Public Relations to Company Name City State Directed the college s media relations activities including the production and dissemination of press releases media advisorie

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Define the job titles and their corresponding labels
job_titles = ['Data Scientist', 'Software Engineer', 'Marketing Manager', 'Engineer']
labels = [1, 2, 3, 4]

In [28]:
# Function to encode a resume text into a BERT-compatible format
def encode_resume(resume_text):
    input_ids = torch.tensor(tokenizer.encode(resume_text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    input_ids = torch.resize(input_ids, (1, 936))
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs[0]
    return logits, input_ids

In [29]:
# Function to classify a resume based on its job title
def classify_resume(resume_text):
    logits,_ = encode_resume(resume_text)
    probs = torch.softmax(logits,dim=1)
    top_prob, top_label = probs.max(1)
    predicted_label = job_titles[top_label.item()]
    return predicted_label

In [31]:
# Example usage
# _,input_ids = encode_resume(text)

input_ids = torch.resize(input_ids, (1, 936))

predicted_label = classify_resume(text)

AttributeError: module 'torch' has no attribute 'resize'

In [ ]:
print("Predicted job title:", predicted_label)  # Output